In [1]:
import pandas as pd
import json

from utils.plotly_theme import *

In [2]:
import json

with open("data/input/equipment_data.json", "r") as f:
    data = json.load(f)
    eq_types = [item["eq_type"] for item in data["equipment"]]

equipment_types = list(set(eq_types))
print(equipment_types)

['boiler', 'heat_pump', 'chiller', 'hr_heat_pump', 'resistance_heater']


In [3]:
# test interpolation

from utils.interp import interp_vector

fp = [3.815, 3.955, 4.133, 3.74] # cop
xp = [1465000, 1098750, 732500, 366250] # cap
x = [1465200]
print(interp_vector(xp, fp, x))

[3.74]


In [4]:
# Example usage of the decarb-tool

from src.metadata import Metadata
from utils.inputs import get_city_and_zone

# Load location data
location_data = pd.read_csv("data/input/locations.csv")
zip_code = "11229" # Brooklyn, NY -> will be replaced by user input from UI (search by zip-code or city name)
locations = get_city_and_zone(zip_code, location_data)

# Based on zip-code, SET location, climate_zone, time_zone
location = locations.city
ashrae_climate_zone = locations.ashrae_climate_zone
time_zone = locations.time_zone

# Set building type, building vintage, load type and equipment via equipment scenario -> eventually replaced by user input from UI
building_type = "Hospital"
vintage = 2004
load_type = "load_simulated"
equipment_scenarios = ["baseline_01", "decarb_01", "typ_01", "typ_02"] # list of equipment scenarios to compare
emission_scenario = "MidCase"

# create Metadata object to store settings (rest of settings are set by default if not specified)
settings = Metadata.create(
    location=location,
    ashrae_climate_zone=ashrae_climate_zone,
    building_type=building_type,
    vintage=vintage,
    load_type=load_type,
    equipment_scenarios=equipment_scenarios
)
settings.emissions.years = [2025, 2040]
settings.emissions.emission_scenario = emission_scenario
settings.emissions.gea_grid_region = "NYISO"
settings.emissions.time_zone = time_zone

print(settings)

location='New York' building_type='Hospital' vintage=2004 load_type='load_simulated' ashrae_climate_zone='4A' equipment_scenarios=['baseline_01', 'decarb_01', 'typ_01', 'typ_02'] emissions=EmissionsSettings(emission_scenario='MidCase', gea_grid_region='NYISO', time_zone='America/New_York', emission_type='Combustion only', shortrun_weighting=1.0, years=[2025, 2040]) units='SI' last_updated='2025-09-23T18:08:19.127897'


In [5]:
max(settings.emissions.years)

2040

In [6]:
# Get datasets based on input (location, load_type) saved in settings object
from src.loads import get_load_data
from src.emissions import get_emissions_data

# get load data
load_data = get_load_data(settings)

# get emission data
emissions_data = get_emissions_data(settings)

# inspect load data
load_data.df.head(30)

/Users/toby/Desktop/Repositories/decarb-tool/src/loads.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=False)


,t_out_C,heating_W,cooling_W
timestamp,,,
2001-01-01 01:00:00,-4.400000,609230.1302,127902.411800
2001-01-01 02:00:00,-4.400000,606900.3412,174577.649100
2001-01-01 03:00:00,-4.108333,604168.5347,186763.682700
2001-01-01 04:00:00,-3.550000,600461.6520,199730.652500
2001-01-01 05:00:00,-3.300000,604702.9970,205553.300800
2001-01-01 06:00:00,-3.650000,607617.8109,200142.297400
2001-01-01 07:00:00,-4.191667,611215.7286,191604.123600
2001-01-01 08:00:00,-3.758333,607994.1257,113827.739800
2001-01-01 09:00:00,-3.008333,546663.1432,109587.775600


In [7]:
# inspect emissions data
emissions_data.df.head(30)

,emission_scenario,gea_grid_region,time_zone,emission_type,year,lrmer_co2e_c,lrmer_co2e_p,srmer_co2e_c,srmer_co2e_p
timestamp,,,,,,,,,
2025-01-01 00:00:00,MidCase,NYISO,America/New_York,Combustion only,2025,213.4,60.9,549.9,156.2
2025-01-01 01:00:00,MidCase,NYISO,America/New_York,Combustion only,2025,209.2,59.1,539.1,159.6
2025-01-01 02:00:00,MidCase,NYISO,America/New_York,Combustion only,2025,198.9,58.7,546.2,159.4
2025-01-01 03:00:00,MidCase,NYISO,America/New_York,Combustion only,2025,180.1,55.9,533.4,160.6
2025-01-01 04:00:00,MidCase,NYISO,America/New_York,Combustion only,2025,176.1,53.6,524.6,163.9
2025-01-01 05:00:00,MidCase,NYISO,America/New_York,Combustion only,2025,194.6,57.2,556.0,168.0
2025-01-01 06:00:00,MidCase,NYISO,America/New_York,Combustion only,2025,240.0,65.5,659.2,168.0
2025-01-01 07:00:00,MidCase,NYISO,America/New_York,Combustion only,2025,233.5,65.9,678.9,163.2
2025-01-01 08:00:00,MidCase,NYISO,America/New_York,Combustion only,2025,220.4,62.4,662.7,170.7


In [8]:
# Test-load equipment library (not changing the actual sim settings)

from src.equipment import load_library

library = load_library("data/input/equipment_data.json")

hp_a = library.get_equipment("hp01")
print(hp_a)

scenario_a = library.get_scenario("decarb_01")
print(scenario_a)

eq_id='hp01' eq_type='heat_pump' eq_subtype='air_to_water' eq_manufacturer=None model='GenericAWHP' fuel='electricity' capacity_W=None performance=Performance(cop_curve=COPCurve(t_out_C=[-10.0, 0.0, 10.0, 20.0, 30.0], cop=[2.0, 2.5, 3.2, 3.8, 4.2]), cap_curve=CapCurve(t_out_C=[-10.0, 0.0, 10.0, 20.0, 30.0], capacity_W=[100000.0, 120000.0, 140000.0, 160000.0, 180000.0]), plr_curve=None, efficiency=None, constraints=None) emissions=None
eq_scen_id='decarb_01' eq_scen_name='Decarbonized HP Only' hr_wwhp='hr01' awhp_h='hp01' awhp_c='hp01' awhp_sizing=4.0 boiler=None chiller='ch01' resistance_heater='res01'


In [9]:
from src.energy import loads_to_site_energy

site_energy = loads_to_site_energy(load_data, library, settings.equipment_scenarios, detail=True)

site_energy.head(30)


,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,awhp_cop_c,awhp_chw_W,elec_awhp_c_Wh,chiller_cop,chiller_chw_W,elec_chiller_Wh,scenario_id,max_cap_h_hr_W,min_cap_h_hr_W,simult_h_hr_W
timestamp,,,,,,,,,,,,,,,,,,,,,
2001-01-01 01:00:00,-4.40,609230.13,127902.41,101719.46,579104.80,609230.13,127902.41,NaN,NaN,NaN,...,2.28,111200.00,48771.93,4.0,16702.41,4175.60,baseline_01,NaN,NaN,NaN
2001-01-01 02:00:00,-4.40,606900.34,174577.65,113388.27,576395.75,606900.34,174577.65,NaN,NaN,NaN,...,2.28,111200.00,48771.93,4.0,63377.65,15844.41,baseline_01,NaN,NaN,NaN
2001-01-01 03:00:00,-4.11,604168.53,186763.68,116177.45,572540.93,604168.53,186763.68,NaN,NaN,NaN,...,2.29,111783.33,48716.18,4.0,74980.35,18745.09,baseline_01,NaN,NaN,NaN
2001-01-01 04:00:00,-3.55,600461.65,199730.65,118930.48,566932.15,600461.65,199730.65,NaN,NaN,NaN,...,2.32,112900.00,48611.41,4.0,86830.65,21707.66,baseline_01,NaN,NaN,NaN
2001-01-01 05:00:00,-3.30,604703.00,205553.30,120168.95,571282.55,604703.00,205553.30,NaN,NaN,NaN,...,2.34,113400.00,48565.31,4.0,92153.30,23038.33,baseline_01,NaN,NaN,NaN
2001-01-01 06:00:00,-3.65,607617.81,200142.30,119120.55,575485.83,607617.81,200142.30,NaN,NaN,NaN,...,2.32,112700.00,48629.99,4.0,87442.30,21860.57,baseline_01,NaN,NaN,NaN
2001-01-01 07:00:00,-4.19,611215.73,191604.12,117460.94,580929.14,611215.73,191604.12,NaN,NaN,NaN,...,2.29,111616.67,48732.04,4.0,79987.46,19996.86,baseline_01,NaN,NaN,NaN
2001-01-01 08:00:00,-3.76,607994.13,113827.74,97636.52,576175.34,607994.13,113827.74,NaN,NaN,NaN,...,2.31,112483.33,48650.21,4.0,1344.41,336.10,baseline_01,NaN,NaN,NaN
2001-01-01 09:00:00,-3.01,546663.14,109587.78,95153.51,503116.06,546663.14,109587.78,NaN,NaN,NaN,...,2.35,109587.78,46641.37,4.0,0.00,0.00,baseline_01,NaN,NaN,NaN


In [10]:
from src.energy import site_to_source

source_energy = site_to_source(site_energy, settings=settings.emissions, emissions=emissions_data)

source_energy.head(30)


,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,scenario_id,max_cap_h_hr_W,min_cap_h_hr_W,simult_h_hr_W,elec_emissions_rate,emission_year,elec_emissions,gas_emissions,total_refrig_emissions,year
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-01-01 01:00:00,-4.40,609230.13,127902.41,101719.46,579104.80,609230.13,127902.41,NaN,NaN,NaN,...,baseline_01,NaN,NaN,NaN,374.15,2025,38.058336,104.238864,0.0,2025
2025-01-01 02:00:00,-4.40,606900.34,174577.65,113388.27,576395.75,606900.34,174577.65,NaN,NaN,NaN,...,baseline_01,NaN,NaN,NaN,372.55,2025,42.242800,103.751235,0.0,2025
2025-01-01 03:00:00,-4.11,604168.53,186763.68,116177.45,572540.93,604168.53,186763.68,NaN,NaN,NaN,...,baseline_01,NaN,NaN,NaN,356.75,2025,41.446305,103.057367,0.0,2025
2025-01-01 04:00:00,-3.55,600461.65,199730.65,118930.48,566932.15,600461.65,199730.65,NaN,NaN,NaN,...,baseline_01,NaN,NaN,NaN,350.35,2025,41.667294,102.047787,0.0,2025
2025-01-01 05:00:00,-3.30,604703.00,205553.30,120168.95,571282.55,604703.00,205553.30,NaN,NaN,NaN,...,baseline_01,NaN,NaN,NaN,375.30,2025,45.099407,102.830859,0.0,2025
2025-01-01 06:00:00,-3.65,607617.81,200142.30,119120.55,575485.83,607617.81,200142.30,NaN,NaN,NaN,...,baseline_01,NaN,NaN,NaN,449.60,2025,53.556599,103.587449,0.0,2025
2025-01-01 07:00:00,-4.19,611215.73,191604.12,117460.94,580929.14,611215.73,191604.12,NaN,NaN,NaN,...,baseline_01,NaN,NaN,NaN,456.20,2025,53.585681,104.567245,0.0,2025
2025-01-01 08:00:00,-3.76,607994.13,113827.74,97636.52,576175.34,607994.13,113827.74,NaN,NaN,NaN,...,baseline_01,NaN,NaN,NaN,441.55,2025,43.111405,103.711561,0.0,2025
2025-01-01 09:00:00,-3.01,546663.14,109587.78,95153.51,503116.06,546663.14,109587.78,NaN,NaN,NaN,...,baseline_01,NaN,NaN,NaN,404.05,2025,38.446776,90.560891,0.0,2025


In [11]:
#! Update for mutliple scenarios and years

from src.visuals import plot_energy_breakdown

df = source_energy.copy()

plot_energy_breakdown(df)


In [12]:

from src.visuals import plot_energy_and_emissions

df = source_energy.copy()

year = 2025

plot_energy_and_emissions(df, year)


In [13]:
#! Update for mutliple scenarios

from src.visuals import plot_emissions_heatmap

df = source_energy.copy()

year = 2040

plot_emissions_heatmap(df, year)


In [14]:
from src.visuals import plot_meter_timeseries

df = source_energy.copy()

metadata_cols = ['scenario_id']
elec_cols = ['elec_hr_Wh',"elec_awhp_h_Wh","elec_chiller_Wh", 'elec_awhp_c_Wh', 'elec_res_Wh']
gas_cols = ['gas_boiler_Wh']

df = df[metadata_cols + elec_cols + gas_cols]

year = 2025
scenario_id = "typ_01"

plot_meter_timeseries(df, year=year, scenario_id=scenario_id, stacked=True, include_gas=True)

In [ ]:
from src.visuals import plot_total_emissions_bar

df = source_energy.copy()

plot_total_emissions_bar(df, unit_mode="IP")